In [ ]:
# prompt: import recordlinkage
# import pandas
# Or install if required

try:
  import recordlinkage
except ModuleNotFoundError:
  !pip install recordlinkage
  import recordlinkage

try:
  import pandas as pd
except ModuleNotFoundError:
  !pip install pandas
  import pandas as pd
import numpy as np

In [109]:
df_register = pd.read_excel('public_data_registerP.xlsx')
df_results = pd.read_excel('public_data_resultsP.xlsx')
df_anonymised = pd.read_csv('anonymised_dataP.csv')
df_survey = pd.read_csv('survey_listP.txt', sep='\t')
df_survey.columns = ['name']
#df_register['dob'] = pd.to_datetime(df_register['dob'], format='%d-%m-%Y')
#df_register['age'] = np.floor((pd.to_datetime('today') - df_register['dob']).dt.days / 365.25).astype(int)
#df_anonymised = df_anonymised.rename(columns={"Age": "age"})

In [110]:
df_register.head()

,name,sex,dob,zip,citizenship,marital_status,last_voted
0,"Tellez, Jose",Male,2004-05-27,2100,Denmark,Never married,2
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Never married,1
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Never married,1
3,"el-Rad, Taaj",Male,1999-06-19,2400,Denmark,Never married,2
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Never married,1


In [111]:
# vote_mapping = {
#     0: 'In-person',  # Vote cast on paper
#     1: 'Online',     # Vote cast electronically
#     2: 'Abstained'   # Did not vote
# }

# df_register['evote'] = df_register['last_voted'].map(vote_mapping)

In [112]:
# Find the register rows that match the names in survey_listK
matched_register = df_register[df_register['name'].isin(df_survey['name'])]
matched_register

,name,sex,dob,zip,citizenship,marital_status,last_voted
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Never married,1
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Never married,1
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Never married,1
9,"Grant, Debray",Male,2003-12-03,2100,Denmark,Never married,1
10,"Blackmore, Gabriela",Female,2004-09-18,2100,Denmark,Never married,1
...,...,...,...,...,...,...,...
1447,"el-Shahin, Muzaina",Female,1926-11-02,2300,Denmark,Widowed,0
1454,"Herrera, Addison",Female,1966-07-05,2300,Denmark,Married/separated,0
1460,"al-Basa, Annnees",Male,1947-09-19,2200,Denmark,Married/separated,0
1472,"Roybal, Karla",Female,1952-03-25,2400,Denmark,Married/separated,0


In [113]:
matched_register.loc[:,'citizenship'] = matched_register['citizenship'].apply(
    lambda x: 'Denmark' if x == 'Denmark' else 'Other'
)
marital_mapping = {
    'Never married': 'Not in relationship',
    'Divorced': 'Not in relationship',
    'Married/separated': 'In relationship',
    'Widowed': 'Not in relationship'
}
matched_register.loc[:,'marital_status'] = matched_register['marital_status'].map(marital_mapping)
matched_register.rename(columns={'last_voted':'evote'}, inplace = True)


/tmp/ipykernel_26733/1722265991.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_register.rename(columns={'last_voted':'evote'}, inplace = True)


In [114]:
matched_register

,name,sex,dob,zip,citizenship,marital_status,evote
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Not in relationship,1
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Not in relationship,1
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Not in relationship,1
9,"Grant, Debray",Male,2003-12-03,2100,Denmark,Not in relationship,1
10,"Blackmore, Gabriela",Female,2004-09-18,2100,Denmark,Not in relationship,1
...,...,...,...,...,...,...,...
1447,"el-Shahin, Muzaina",Female,1926-11-02,2300,Denmark,Not in relationship,0
1454,"Herrera, Addison",Female,1966-07-05,2300,Denmark,In relationship,0
1460,"al-Basa, Annnees",Male,1947-09-19,2200,Denmark,In relationship,0
1472,"Roybal, Karla",Female,1952-03-25,2400,Denmark,In relationship,0


In [115]:
matched_register['zip'].value_counts()

zip
2300    65
2400    59
2200    41
2100    34
Name: count, dtype: int64

In [116]:
df_anonymised['zip'].value_counts()

zip
0    65
2    57
1    41
3    33
Name: count, dtype: int64

In [117]:
###Trying to see if I can deanonymize zip codes
###IDK if correct
zip_mapping = {
    2100 : 3,
    2200 : 1,
    2300 : 0,
    2400 : 2
}
matched_register.loc[:,'mapped_zip'] = matched_register['zip'].map(zip_mapping)
matched_register

/tmp/ipykernel_26733/2949012234.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_register.loc[:,'mapped_zip'] = matched_register['zip'].map(zip_mapping)


,name,sex,dob,zip,citizenship,marital_status,evote,mapped_zip
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Not in relationship,1,0
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Not in relationship,1,3
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Not in relationship,1,1
9,"Grant, Debray",Male,2003-12-03,2100,Denmark,Not in relationship,1,3
10,"Blackmore, Gabriela",Female,2004-09-18,2100,Denmark,Not in relationship,1,3
...,...,...,...,...,...,...,...,...
1447,"el-Shahin, Muzaina",Female,1926-11-02,2300,Denmark,Not in relationship,0,0
1454,"Herrera, Addison",Female,1966-07-05,2300,Denmark,In relationship,0,0
1460,"al-Basa, Annnees",Male,1947-09-19,2200,Denmark,In relationship,0,1
1472,"Roybal, Karla",Female,1952-03-25,2400,Denmark,In relationship,0,2


In [ ]:
indexer = recordlinkage.Index()
indexer.full()
#indexer.block('sex')
#indexer.block('citizenship')
#indexer.block('marital_status')
# indexer.block('evote')
# indexer.block('age')

candidate_links = indexer.index(matched_register, df_anonymised)
candidate_links

MultiIndex([(   1,   0),
            (   1,   1),
            (   1,   2),
            (   1,   3),
            (   1,   4),
            (   1,   5),
            (   1,   6),
            (   1,   7),
            (   1,   8),
            (   1,   9),
            ...
            (1475, 186),
            (1475, 187),
            (1475, 188),
            (1475, 189),
            (1475, 190),
            (1475, 191),
            (1475, 192),
            (1475, 193),
            (1475, 194),
            (1475, 195)],
           length=39004)

In [119]:
compare = recordlinkage.Compare()

compare.exact('sex', 'sex')
compare.exact('citizenship', 'citizenship')
compare.exact('marital_status', 'marital_status')
compare.exact('evote', 'evote')
compare.exact('mapped_zip','zip')
#compare.numeric('age', 'age')

# The comparison vectors
compare_vectors = compare.compute(candidate_links, matched_register, df_anonymised)

In [120]:
print(compare_vectors)

          0  1  2  3  4
1    0    0  0  0  0  0
     1    1  1  0  0  0
     2    0  1  0  0  0
     3    1  1  0  1  1
     4    1  1  1  0  0
...      .. .. .. .. ..
1475 191  0  1  0  1  0
     192  0  1  1  0  1
     193  1  1  1  1  0
     194  1  1  0  1  1
     195  0  1  0  0  0

[39004 rows x 5 columns]


In [121]:
matches = compare_vectors[compare_vectors.sum(axis=1) >= 5]
matches

0  1  2  3  4
1    10   1  1  1  1  1
     17   1  1  1  1  1
     22   1  1  1  1  1
     25   1  1  1  1  1
     29   1  1  1  1  1
...      .. .. .. .. ..
1475 110  1  1  1  1  1
     111  1  1  1  1  1
     120  1  1  1  1  1
     130  1  1  1  1  1
     180  1  1  1  1  1

[1430 rows x 5 columns]

In [122]:
# Save matches to CSV
matches.to_csv('matches.csv')
match_df = pd.read_csv('matches.csv')
match_df.columns = ['public_idx', 'an_idx', '0', '1', '2', '3','4']
match_df.head()

,public_idx,an_idx,0,1,2,3,4
0,1,10,1,1,1,1,1
1,1,17,1,1,1,1,1
2,1,22,1,1,1,1,1
3,1,25,1,1,1,1,1
4,1,29,1,1,1,1,1


In [123]:
grouped = match_df.groupby('public_idx')

results = {}

for public_idx, group in grouped:
  name = df_register.loc[public_idx, 'name']

  anonymised_indices = group['an_idx']
  parties = df_anonymised.loc[anonymised_indices, 'party'].tolist()

  if len(parties) == 1:
    party = parties[0]
    results[name] = party
  else:
    if len(set(parties)) == 1:
      party = parties[0]
      results[name] = party

results_df = pd.DataFrame(list(results.items()), columns=['name', 'party'])
print(results_df)

results_df.to_csv('final_matches.csv')

                      name  party
0     Chandana, Keosomalee  Green
1      Blackmore, Gabriela  Green
2         Todacheene, Sara  Green
3           Figgs, Preston  Green
4       Flanagan, Danielle  Green
5          Lechuga, Hernan  Green
6       Wu, Matthew-Angelo  Green
7               Mora, Anna  Green
8            Kallon, Kaire  Green
9      Arias Torres, Dulce  Green
10       al-Usman, Taqiyya  Green
11          Golden, Jerred  Green
12         Molina, Jasmine    Red
13           Mason, Darrin  Green
14           Alferez, Yeen  Green
15           Dwyer, Amanda    Red
16         Aguilar, Damien    Red
17          Dinsmore, Seth  Green
18            Forney, Amir  Green
19         Fisher, Addison  Green
20          Tillman, Ocean  Green
21               Ta, Saqib  Green
22          Murtari, Yoshi  Green
23       Romero Diaz, Alec    Red
24           Yuan, Anirudh  Green
25     Stufflebean, Garitt  Green
26           Walker, Natae  Green
27            Dauer, Karan  Green
28     Jacobs,

To do: merge parties with impartial matches and see if there are records of people who dont exactly match demographics but all have voted for the same party(l-diversity=1)

In [124]:
quasi_matches = compare_vectors[compare_vectors.sum(axis=1) == 4]
quasi_matches

0  1  2  3  4
1    3    1  1  0  1  1
     11   0  1  1  1  1
     12   0  1  1  1  1
     26   0  1  1  1  1
     41   1  1  1  0  1
...      .. .. .. .. ..
1475 182  1  1  1  1  0
     184  1  1  1  1  0
     186  1  1  0  1  1
     193  1  1  1  1  0
     194  1  1  0  1  1

[7322 rows x 5 columns]

In [125]:
matched_register

,name,sex,dob,zip,citizenship,marital_status,evote,mapped_zip
1,"Donlin, Jacob",Male,2004-03-30,2300,Denmark,Not in relationship,1,0
2,"Chandana, Keosomalee",Female,2001-09-20,2100,Denmark,Not in relationship,1,3
4,"Sun, Stephanie",Female,2002-04-23,2200,Denmark,Not in relationship,1,1
9,"Grant, Debray",Male,2003-12-03,2100,Denmark,Not in relationship,1,3
10,"Blackmore, Gabriela",Female,2004-09-18,2100,Denmark,Not in relationship,1,3
...,...,...,...,...,...,...,...,...
1447,"el-Shahin, Muzaina",Female,1926-11-02,2300,Denmark,Not in relationship,0,0
1454,"Herrera, Addison",Female,1966-07-05,2300,Denmark,In relationship,0,0
1460,"al-Basa, Annnees",Male,1947-09-19,2200,Denmark,In relationship,0,1
1472,"Roybal, Karla",Female,1952-03-25,2400,Denmark,In relationship,0,2


In [126]:
df_anonymised['an_idx'] = df_anonymised.index
df_anonymised

,sex,evote,zip,education,citizenship,marital_status,party,age,an_idx
0,Female,0,1,University,Other,In relationship,Red,40-59,0
1,Male,0,3,University,Denmark,In relationship,Red,40-59,1
2,Female,0,1,University,Denmark,In relationship,Green,40-59,2
3,Male,1,0,University,Denmark,In relationship,Green,40-59,3
4,Male,0,3,University,Denmark,Not in relationship,Green,40-59,4
...,...,...,...,...,...,...,...,...,...
191,Male,0,1,Other,Denmark,Not in relationship,Green,40-59,191
192,Male,1,0,Other,Denmark,In relationship,Red,40-59,192
193,Female,0,1,University,Denmark,In relationship,Red,40-59,193
194,Female,0,0,University,Denmark,Not in relationship,Green,40-59,194


In [127]:
# Save matches to CSV
quasi_matches.to_csv('quasi_matches.csv')
quasi_match_df = pd.read_csv('quasi_matches.csv')
quasi_match_df.columns = ['public_idx', 'an_idx', '0', '1', '2', '3','4']
#uasi_match_df.join(df_anonymised['party'], on='an_idx',lsuffix='_caller', rsuffix='_other')
quasi_match_df.merge(df_anonymised, how='left', on='an_idx')
quasi_match_df #size = 7322x7

,public_idx,an_idx,0,1,2,3,4
0,1,3,1,1,0,1,1
1,1,11,0,1,1,1,1
2,1,12,0,1,1,1,1
3,1,26,0,1,1,1,1
4,1,41,1,1,1,0,1
...,...,...,...,...,...,...,...
7317,1475,182,1,1,1,1,0
7318,1475,184,1,1,1,1,0
7319,1475,186,1,1,0,1,1
7320,1475,193,1,1,1,1,0


In [128]:
quasi_match_df

,public_idx,an_idx,0,1,2,3,4
0,1,3,1,1,0,1,1
1,1,11,0,1,1,1,1
2,1,12,0,1,1,1,1
3,1,26,0,1,1,1,1
4,1,41,1,1,1,0,1
...,...,...,...,...,...,...,...
7317,1475,182,1,1,1,1,0
7318,1475,184,1,1,1,1,0
7319,1475,186,1,1,0,1,1
7320,1475,193,1,1,1,1,0
